In [8]:
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.filters import sobel
from scipy.stats import entropy
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy

def extract_optical_flow(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    optical_flows = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if prev_frame is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_frame, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            flow_magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            optical_flows.append(np.mean(flow_magnitude))
        prev_frame = gray_frame

    cap.release()
    return downsample_to_1hz(optical_flows, fps)

def extract_hog_features(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    hog_features = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = rgb2gray(frame)
        hog_feature = hog(gray_frame, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False)
        hog_features.append(np.mean(hog_feature))

    cap.release()
    return downsample_to_1hz(hog_features, fps)

def extract_color_histogram(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    color_histograms = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        hist_b = cv2.calcHist([frame], [0], None, [256], [0, 256])
        hist_g = cv2.calcHist([frame], [1], None, [256], [0, 256])
        hist_r = cv2.calcHist([frame], [2], None, [256], [0, 256])
        color_hist = np.concatenate((hist_b, hist_g, hist_r)).flatten()
        color_histograms.append(np.mean(color_hist))

    cap.release()
    return downsample_to_1hz(color_histograms, fps)

def extract_edge_detection(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    edge_intensities = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = sobel(gray_frame)
        edge_intensity = np.mean(edges)
        edge_intensities.append(edge_intensity)

    cap.release()
    return downsample_to_1hz(edge_intensities, fps)

def extract_scene_cuts(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    prev_hist = None
    scene_cuts = []
    cuts_per_second = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray_frame], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()

        if prev_hist is not None:
            diff = np.sum(np.abs(hist - prev_hist))
            if diff > 0.4:  # Threshold to detect scene cuts
                cuts_per_second += 1

        prev_hist = hist

        if cap.get(cv2.CAP_PROP_POS_FRAMES) % fps == 0:
            scene_cuts.append(cuts_per_second)
            cuts_per_second = 0

    cap.release()
    return scene_cuts

def extract_saliency(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    saliency_scores = []

    saliency = cv2.saliency.StaticSaliencyFineGrained_create()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        success, saliency_map = saliency.computeSaliency(frame)
        saliency_score = np.mean(saliency_map)
        saliency_scores.append(saliency_score)

    cap.release()
    return downsample_to_1hz(saliency_scores, fps)

def extract_temporal_smoothness(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    smoothness_values = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if prev_frame is not None:
            diff = np.mean(np.abs(gray_frame - prev_frame))
            smoothness_values.append(diff)
        prev_frame = gray_frame

    cap.release()
    return downsample_to_1hz(smoothness_values, fps)

def extract_flicker_brightness(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    brightness_flickers = []
    prev_brightness = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        brightness = np.mean(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        if prev_brightness is not None:
            flicker = abs(brightness - prev_brightness)
            brightness_flickers.append(flicker)
        prev_brightness = brightness

    cap.release()
    return downsample_to_1hz(brightness_flickers, fps)

def extract_spectral_entropy(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    spectral_entropies = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fourier_transform = np.fft.fft2(gray_frame)
        magnitude_spectrum = np.abs(fourier_transform)
        spectral_entropy = shannon_entropy(magnitude_spectrum)
        spectral_entropies.append(spectral_entropy)

    cap.release()
    return downsample_to_1hz(spectral_entropies, fps)

def extract_spatial_frequency(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    spatial_frequencies = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frequency_spectrum = np.fft.fftshift(np.fft.fft2(gray_frame))
        magnitude_spectrum = np.abs(frequency_spectrum)
        spatial_frequency = np.mean(magnitude_spectrum)
        spatial_frequencies.append(spatial_frequency)

    cap.release()
    return downsample_to_1hz(spatial_frequencies, fps)

def extract_luminance_contrast(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    luminance_values = []
    contrast_values = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        luminance = np.mean(gray_frame)
        contrast = gray_frame.std()
        luminance_values.append(luminance)
        contrast_values.append(contrast)

    cap.release()

    luminance_downsampled = downsample_to_1hz(luminance_values, fps)
    contrast_downsampled = downsample_to_1hz(contrast_values, fps)

    return luminance_downsampled, contrast_downsampled

def extract_texture_features(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    texture_contrasts = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        glcm = graycomatrix(gray_frame, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
        texture_contrast = graycoprops(glcm, 'contrast')[0, 0]
        texture_contrasts.append(texture_contrast)

    cap.release()
    return downsample_to_1hz(texture_contrasts, fps)

def downsample_to_1hz(feature_values, fps):
    """Downsamples the feature values to 1Hz (1 value per second)."""
    return [np.mean(feature_values[i:i + fps]) for i in range(0, len(feature_values), fps)]

def process_video_features(video_path):
    fps = 60  # Adjust according to the video fps
    features = {
        "optical_flow": extract_optical_flow(video_path, fps),
        "hog_features": extract_hog_features(video_path, fps),
        "color_histogram": extract_color_histogram(video_path, fps),
        "optical_flow_entropy": extract_optical_flow(video_path, fps),
        "edge_intensity": extract_edge_detection(video_path, fps),
        "scene_cuts": extract_scene_cuts(video_path, fps),
        "temporal_smoothness": extract_temporal_smoothness(video_path, fps),
        "brightness_flicker": extract_flicker_brightness(video_path, fps),
        "spectral_entropy": extract_spectral_entropy(video_path, fps),
        "spatial_frequency": extract_spatial_frequency(video_path, fps),
        "luminance": extract_luminance_contrast(video_path, fps)[0],
        "contrast": extract_luminance_contrast(video_path, fps)[1],
        "texture_contrast": extract_texture_features(video_path, fps)
    }

    return pd.DataFrame(features)

# Example Usage
video_file = '14_video.mkv'  # Replace with your video file path
df = process_video_features(video_file)
df = df.head(840)
print(df.head())
df.to_csv("14_video_features.csv")



In [7]:
saliency = cv2.saliency.StaticSaliencyFineGrained_create()

AttributeError: module 'cv2' has no attribute 'saliency'